# Bittensor RPC Exploration for Stake Tracking

This notebook explores various RPC methods available in the Bittensor network to track:
1. Current stake values
2. Stake transactions (add_stake/remove_stake)
3. Epoch boundaries

The goal is to determine the most reliable methods to use in the vesting system's stake tracker.

In [28]:
# Import necessary libraries
import asyncio
import json
from datetime import datetime, timezone, timedelta
import time
import pandas as pd
import matplotlib.pyplot as plt
from substrateinterface import SubstrateInterface
import bittensor as bt
import nest_asyncio
nest_asyncio.apply()

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

## 1. Connect to Bittensor Network

In [29]:
# Connect to Bittensor
def connect_substrate():
    """Connect to Bittensor and return a SubstrateInterface"""
    endpoint = "wss://entrypoint-finney.opentensor.ai:443"
    print(f"Connecting to endpoint: {endpoint}")
    
    substrate = SubstrateInterface(
        url=endpoint,
    )
    
    # Test connection by getting current block
    current_block = substrate.get_block_number(substrate.get_chain_head())
    print(f"Connected successfully. Current block: {current_block}")
    
    return substrate

substrate = connect_substrate()

Connecting to endpoint: wss://entrypoint-finney.opentensor.ai:443
Connected successfully. Current block: 5055820


## 2. Utility Functions for RPC Calls

In [30]:
# Utility for running RPC calls
async def run_rpc_test(method, params=None):
    """Test an RPC method and print results"""
    if params is None:
        params = []
    try:
        result = await asyncio.to_thread(
            substrate.rpc_request, 
            method, 
            params
        )
        return result
    except Exception as e:
        print(f"Error calling {method} with params {params}: {e}")
        return {"error": str(e)}

# Define helper to run async functions
def run_async(coro):
    """Run an async coroutine"""
    return asyncio.run(coro)

# Pretty print RPC result
def print_rpc_result(result, title=None):
    """Pretty print RPC result"""
    if title:
        print(f"\n=== {title} ===\n")
    
    if isinstance(result, dict) and "result" in result:
        if isinstance(result["result"], bytes):
            print(f"Result is binary data of length {len(result['result'])} bytes")
            # Try to decode as utf-8
            try:
                decoded = result["result"].decode('utf-8')
                print(f"UTF-8 decoded: {decoded[:100]}..." if len(decoded) > 100 else decoded)
            except:
                print("Could not decode as UTF-8")
        else:
            print(json.dumps(result["result"], indent=2))
    else:
        print(json.dumps(result, indent=2))

## 3. Discover Available RPC Methods

In [31]:
# Get available RPC methods
available_methods = run_async(run_rpc_test("rpc_methods"))
print_rpc_result(available_methods, "Available RPC Methods")

# Extract methods related to subnet, neuron, and stake
if "result" in available_methods and "methods" in available_methods["result"]:
    methods = available_methods["result"]["methods"]
    filtered_methods = [
        method for method in methods 
        if any(keyword in method.lower() for keyword in ["subnet", "neuron", "stake", "validator"])
    ]
    print("\nFiltered methods related to subnets, neurons, and stake:")
    for method in filtered_methods:
        print(f"- {method}")


=== Available RPC Methods ===

{
  "methods": [
    "account_nextIndex",
    "author_hasKey",
    "author_hasSessionKeys",
    "author_insertKey",
    "author_pendingExtrinsics",
    "author_removeExtrinsic",
    "author_rotateKeys",
    "author_submitAndWatchExtrinsic",
    "author_submitExtrinsic",
    "author_unwatchExtrinsic",
    "chainHead_v1_body",
    "chainHead_v1_call",
    "chainHead_v1_continue",
    "chainHead_v1_follow",
    "chainHead_v1_header",
    "chainHead_v1_stopOperation",
    "chainHead_v1_storage",
    "chainHead_v1_unfollow",
    "chainHead_v1_unpin",
    "chainSpec_v1_chainName",
    "chainSpec_v1_genesisHash",
    "chainSpec_v1_properties",
    "chain_getBlock",
    "chain_getBlockHash",
    "chain_getFinalisedHead",
    "chain_getFinalizedHead",
    "chain_getHead",
    "chain_getHeader",
    "chain_getRuntimeVersion",
    "chain_subscribeAllHeads",
    "chain_subscribeFinalisedHeads",
    "chain_subscribeFinalizedHeads",
    "chain_subscribeNewHead",
    "

## 4. Get Subnet Information

In [32]:
# Get subnet information
subnets_info_result = run_async(run_rpc_test("subnetInfo_getSubnetsInfo"))
print_rpc_result(subnets_info_result, "Subnets Info")

# Select a subnet for further exploration
# We'll use subnet 1 (typically the main Bittensor network) by default
NETUID = 30  # Change this to explore different subnets

# Get detailed info for the selected subnet
subnet_info_result = run_async(run_rpc_test("subnetInfo_getSubnetInfo", [NETUID]))
print_rpc_result(subnet_info_result, f"Subnet {NETUID} Info")


=== Subnets Info ===

[
  33,
  1,
  1,
  0,
  40,
  254,
  255,
  1,
  0,
  19,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  63,
  1,
  64,
  1,
  1,
  0,
  254,
  255,
  3,
  0,
  200,
  1,
  1,
  1,
  1,
  225,
  6,
  145,
  1,
  0,
  0,
  0,
  130,
  132,
  30,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  4,
  40,
  254,
  255,
  1,
  0,
  11,
  114,
  251,
  94,
  57,
  190,
  66,
  129,
  112,
  1,
  2,
  32,
  254,
  255,
  3,
  0,
  200,
  1,
  16,
  1,
  16,
  92,
  141,
  1,
  4,
  0,
  0,
  46,
  22,
  140,
  8,
  226,
  238,
  117,
  234,
  17,
  228,
  197,
  183,
  245,
  218,
  194,
  231,
  53,
  39,
  140,
  250,
  11,
  21,
  144,
  201,
  133,
  102,
  144,
  246,
  102,
  83,
  189,
  216,
  91,
  112,
  145,
  4,
  1,
  8,
  40,
  254,
  255,
  1,
  0,
  19,
  54,
  96,
  107,
  60,
  51,
  173,
  48,
  2,
  33,
  78,
  1,
  1,
  4,


## 5. Get Neurons in the Subnet

In [33]:
# Get neurons in the subnet
neurons_result = run_async(run_rpc_test("neuronInfo_getNeuronsLite", [NETUID]))
print(f"Got neurons response with size: {len(str(neurons_result))}")

# Select a neuron UID for further exploration
# We'll use UID 0 by default (typically one of the first miners)
UID = 30  # Change this to explore different neurons

# Get detailed info for a specific neuron
neuron_info_result = run_async(run_rpc_test("neuronInfo_getNeuronLite", [NETUID, UID]))
print_rpc_result(neuron_info_result, f"Neuron {UID} Info")

# Extract hotkey for further queries
HOTKEY = "5C4hrfjw9DjXZTzV3MwzrrAr9P1MJhSrvWGWqi1eSuyUpnhM"  # Placeholder - update from results if possible

Got neurons response with size: 197632

=== Neuron 30 Info ===

[
  1,
  206,
  92,
  82,
  128,
  183,
  68,
  15,
  213,
  209,
  240,
  229,
  66,
  136,
  195,
  113,
  208,
  112,
  40,
  163,
  13,
  98,
  21,
  192,
  76,
  54,
  155,
  15,
  35,
  0,
  243,
  154,
  110,
  172,
  193,
  240,
  210,
  212,
  182,
  84,
  101,
  206,
  5,
  207,
  50,
  171,
  70,
  244,
  154,
  75,
  132,
  190,
  237,
  44,
  100,
  217,
  167,
  117,
  170,
  118,
  54,
  116,
  77,
  134,
  83,
  120,
  120,
  0,
  92,
  26,
  75,
  0,
  0,
  0,
  0,
  0,
  64,
  84,
  137,
  0,
  41,
  188,
  14,
  72,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  21,
  39,
  4,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  4,
  172,
  193,
  240,
  210,
  212,
  182,
  84,
  101,
  206,
  5,
  207,
  50,
  171,
  70,
  244,
  154,
  75,
  132,
  190,
  237,
  44,
  1

## 6. Explore Stake Information Methods

In [38]:
# Try different stake-related RPC methods

# 1. Try stakeInfo_getStake if available
stake_info_result = run_async(run_rpc_test("stakeInfo_getStake", [NETUID, HOTKEY]))
print_rpc_result(stake_info_result, "Stake Info from stakeInfo_getStake")

# 2. Try stakeInfo_getStakes if available
stakes_info_result = run_async(run_rpc_test("stakeInfo_getStakes", [NETUID]))
print(f"Got stakes response with size: {len(str(stakes_info_result))}")

# 3. Try getWeightsForValidator if available
validator_weights_result = run_async(run_rpc_test("getWeightsForValidator", [NETUID, UID]))
print_rpc_result(validator_weights_result, "Weights for Validator")

Error calling stakeInfo_getStake with params [30, '5C4hrfjw9DjXZTzV3MwzrrAr9P1MJhSrvWGWqi1eSuyUpnhM']: EOF occurred in violation of protocol (_ssl.c:2426)

=== Stake Info from stakeInfo_getStake ===

{
  "error": "EOF occurred in violation of protocol (_ssl.c:2426)"
}
Error calling stakeInfo_getStakes with params [30]: [SSL: BAD_LENGTH] bad length (_ssl.c:2426)
Got stakes response with size: 55
Error calling getWeightsForValidator with params [30, 30]: [SSL: BAD_LENGTH] bad length (_ssl.c:2426)

=== Weights for Validator ===

{
  "error": "[SSL: BAD_LENGTH] bad length (_ssl.c:2426)"
}


## 7. Explore Block Processing for Stake Transactions

In [39]:
# Get current block
current_block = substrate.get_block_number(substrate.get_chain_head())
print(f"Current block: {current_block}")

# Number of blocks to scan for stake transactions
blocks_to_scan = 1000  # Adjust based on how far back you want to look

# Collect stake transactions
stake_transactions = []

# Scan a range of recent blocks
for block_num in range(current_block - blocks_to_scan, current_block + 1, 100):
    print(f"Processing block {block_num}...")
    try:
        block_hash = substrate.get_block_hash(block_num)
        block = substrate.get_block(block_hash)
        
        for extrinsic_idx, extrinsic in enumerate(block['extrinsics']):
            if 'call' in extrinsic and 'call_module' in extrinsic['call']:
                if extrinsic['call']['call_module'] == 'SubtensorModule':
                    if extrinsic['call']['call_function'] in ['add_stake', 'remove_stake']:
                        print(f"Found stake transaction in block {block_num}:")
                        
                        # Extract transaction details
                        tx_details = {
                            'block_number': block_num,
                            'extrinsic_idx': extrinsic_idx,
                            'type': extrinsic['call']['call_function'],
                            'params': {}
                        }
                        
                        # Extract signer (coldkey)
                        if 'signature' in extrinsic and 'address' in extrinsic['signature']:
                            tx_details['coldkey'] = extrinsic['signature']['address']
                        
                        # Extract params
                        for param in extrinsic['call']['call_args']:
                            tx_details['params'][param['name']] = param['value']
                        
                        stake_transactions.append(tx_details)
                        print(json.dumps(tx_details, indent=2))
    except Exception as e:
        print(f"Error processing block {block_num}: {e}")

print(f"\nFound {len(stake_transactions)} stake transactions")

# Create a pandas DataFrame from the transactions
if stake_transactions:
    df = pd.DataFrame(stake_transactions)
    display(df)

SSLError: [SSL: BAD_LENGTH] bad length (_ssl.c:2426)

## 8. Detect Epoch Boundaries

In [36]:
# Get subnet information to analyze epoch boundaries
def check_epoch_boundary(netuid):
    """Check if we're at an epoch boundary by examining subnet state"""
    try:
        # Get subnet info
        result = substrate.rpc_request("subnetInfo_getSubnetInfo", [netuid])
        
        # Examine result to find epoch-related fields
        print(f"\nSubnet info for detecting epochs (netuid {netuid}):")
        if "result" in result:
            print(f"Got binary result of length {len(result['result'])} bytes")
            return result
        else:
            print("No result found")
            return None
            
    except Exception as e:
        print(f"Error checking epoch boundary: {e}")
        return None

epoch_check_result = check_epoch_boundary(NETUID)
print_rpc_result(epoch_check_result, "Epoch Boundary Check")


Subnet info for detecting epochs (netuid 30):
Got binary result of length 72 bytes

=== Epoch Boundary Check ===

[
  1,
  120,
  40,
  154,
  153,
  2,
  0,
  11,
  30,
  77,
  147,
  138,
  183,
  83,
  130,
  81,
  1,
  0,
  1,
  1,
  4,
  254,
  255,
  3,
  0,
  200,
  1,
  4,
  1,
  4,
  188,
  161,
  5,
  0,
  0,
  0,
  130,
  132,
  30,
  0,
  68,
  73,
  48,
  186,
  162,
  250,
  73,
  178,
  57,
  186,
  200,
  145,
  152,
  54,
  164,
  229,
  129,
  0,
  104,
  17,
  112,
  34,
  7,
  188,
  120,
  240,
  34,
  45,
  240,
  122,
  2,
  87
]


## 9. Monitor Stake Changes Over Time

In [37]:
# This cell would be used to monitor stake changes over time
# We'd need to run repeated queries and store the results

# Example monitoring function (pseudo-code, doesn't actually run)
def monitor_stake_changes(netuid, hotkey, interval_seconds=60, duration_minutes=10):
    """Monitor stake changes for a specific hotkey over time"""
    start_time = datetime.now(timezone.utc)
    end_time = start_time + timedelta(minutes=duration_minutes)
    
    stake_history = []
    
    print(f"Starting stake monitoring for hotkey {hotkey} in subnet {netuid}")
    print(f"Will run for {duration_minutes} minutes with {interval_seconds} second intervals")
    
    while datetime.now(timezone.utc) < end_time:
        # Get current stake
        try:
            # Query stake using appropriate method determined from above experiments
            # Record timestamp and stake value
            stake_history.append({
                'timestamp': datetime.now(timezone.utc),
                'stake': 0  # Replace with actual stake value
            })
            
            # Sleep until next interval
            time.sleep(interval_seconds)
            
        except Exception as e:
            print(f"Error monitoring stake: {e}")
            time.sleep(interval_seconds)
    
    # Analyze and visualize results
    if stake_history:
        df = pd.DataFrame(stake_history)
        plt.figure(figsize=(10, 6))
        plt.plot(df['timestamp'], df['stake'])
        plt.title(f"Stake Changes for {hotkey}")
        plt.xlabel("Time")
        plt.ylabel("Stake (TAO)")
        plt.grid(True)
        plt.show()
        
        return df
    else:
        print("No stake history collected")
        return None

# Note: Uncomment the following line to run actual monitoring
# stake_df = monitor_stake_changes(NETUID, HOTKEY)

## 10. Notes and Observations

Document your findings here as you experiment with the RPC methods:

1. Most effective method for getting current stake: ...
2. Most effective way to detect stake transactions: ...
3. Most reliable way to detect epoch boundaries: ...
4. Challenges and limitations: ...
5. Recommended implementation approach: ...

## 11. Summary and Implementation Plan

Based on the experiments in this notebook, outline the implementation plan for the stake tracking system:

### Current Implementation

```python
class RPCTransactionTracker:
    async def query_transactions(self, coldkeys=None, hotkeys=None):
        # Method details to be determined from experiments
        pass

class RPCStakeMonitor:
    async def get_stake(self, hotkey):
        # Method details to be determined from experiments
        pass
        
    async def check_epoch_boundary(self):
        # Method details to be determined from experiments
        pass
```

### Implementation Plan

1. Update `RPCTransactionTracker` to use: ...
2. Update `RPCStakeMonitor` to use: ...
3. Additional utilities needed: ...
4. Performance considerations: ...
5. Error handling approach: ...